<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/0e3cfbdfbcf321681c1ac1c387baf7a1a41d8d21/tools/colab/regression_sine_data_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# A simple regression model using Keras with Cloud TPUs



## Overview

This notebook demonstrates using Cloud TPUs in colab to build a simple regression model using **y = sin(x)** to predict y for given x.

This model generates huge amounts of data that and demonstrates the training performance advantage of Cloud TPU over CPU.

The model trains for 10 epochs with 512 steps per epoc on both TPU and CPU and completes in approximately 10 minutes.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Learning objectives

In this Colab, you will learn how to:
*   Create and compile a Keras model on TPU with a distribution strategy.
*   Train, evaluate, and and generate predictions on Cloud TPU.
*   Compare the performance of a TPU versus a CPU.






## Instructions

<h3>  &nbsp;&nbsp;Train on TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER.

### Imports

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import math
import os
import pprint
import matplotlib.pyplot as plt

from sklearn import metrics

In [0]:
print(tf.__version__)
if tf.__version__.split('.')[1] == '13':
    raise Exception('This notebook is not compatible with Tensorflow 1.13, please use the previous version at https://github.com/tensorflow/tpu/blob/913cf31d85bc31541fbdafa9d0b87db71dd6dcba/tools/colab/regression_sine_data_with_keras.ipynb')

### Resolve TPU Address

In [0]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  print ('List of devices:')
  pprint.pprint(session.list_devices())

### Creating data for y = sin(x).
Sine wave data is created using numpy. And to make it more difficult, random noice is added to the sine wave.

In [0]:
data_size = 2**18

x = np.linspace(0, 6, data_size, dtype=np.float32)
np.random.shuffle(x)

y = -20 * np.sin(x, dtype=np.float32) + 3 + np.random.normal(0, 1, (data_size,)).astype(np.float32)

x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

train_x, test_x = x[:data_size//2], x[data_size//2:]
train_y, test_y = y[:data_size//2], y[data_size//2:]

plt.plot(x, y, 'bo')

### Define model:
Model will have an input layer where it takes in the x coordinate, two densely connected layers with 200 and 80 nodes, and an output layer where it returns the predicted y value.

In [0]:
def get_model():
  return keras.Sequential([
      keras.layers.Dense(1, input_shape=(1,)),
      keras.layers.Dense(200, activation='sigmoid'),
      keras.layers.Dense(80, activation='sigmoid'),
      keras.layers.Dense(1)
  ])

### Compiling the model with a distribution strategy
To make the model usable by a TPU, we first must create and compile it using a distribution strategy.

In [0]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TF_MASTER)
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

In [0]:
with strategy.scope():
  model = get_model()
  model.compile(optimizer=tf.keras.optimizers.SGD(.01),
                loss='mean_squared_error',
                metrics=['mean_squared_error'])

###Training of the model on TPU

In [0]:
model.fit(train_x, train_y, epochs=10, steps_per_epoch=512)
model.save_weights('/tmp/sine.h5', overwrite=True)

###Prediction
For predictions, same model architecture is being used which is loaded with the earlier learned weights.

In [0]:
prediction_model = get_model()
prediction_model.load_weights('/tmp/sine.h5')

In [0]:
predictions = prediction_model.predict(test_x)
plt.plot(test_x, predictions, 'ro')

###Training on CPU for comparison with TPU
How much time did we save by using a TPU? Let's try training the exact same model on the same data, but without the TPU speedup.

**(WARNING: This will take approximately 7 minutes to execute!)**

In [0]:
slow_model = get_model()
slow_model.compile(optimizer=tf.train.GradientDescentOptimizer(.01),
             loss='mean_squared_error',
             metrics=['mean_squared_error'])
slow_model.fit(train_x, train_y, epochs=10, steps_per_epoch=512)

## What's next

* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.

On Google Cloud Platform, in addition to GPUs and TPUs available on pre-configured [deep learning VMs](https://cloud.google.com/deep-learning-vm/),  you will find [AutoML](https://cloud.google.com/automl/)*(beta)* for training custom models without writing code and [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) which will allows you to run parallel trainings and hyperparameter tuning of your custom models on powerful distributed hardware.
